In [1]:
from utils.elpv_reader import load_dataset
import numpy as np
import matplotlib as plt
import cv2
import os

# scikit-learn imports
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

# tensorflow imports
import tensorflow
from tensorflow import keras
from keras.applications import VGG19
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D
from keras import Input
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras import layers

# from PIL import Image
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
images, probs, types = load_dataset()
print(images.shape)
print(probs.shape)
print(types.shape)

(2624, 300, 300)
(2624,)
(2624,)


In [11]:
# OTSU IMAGE SEGMENTATION HUEHUEHUE
# uncomment below code for otsu segmentation
# for i in range(0, len(images)):
#     retOtsu,thOtsu = cv2.threshold(images[i],0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
#     images[i] = thOtsu

# kernelLaplacian = np.array([[1, 1, 1],
#                             [1, -8, 1],
#                             [1, 1, 1]])

# for i in range(0, len(images)):
#     norm_img = cv2.normalize(images[i], None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
#     norm_img = (255*norm_img).astype(np.uint8)
#     images[i] = norm_img
#     # edging = cv2.filter2D(src=norm_img, ddepth=-1, kernel=kernelLaplacian)
#     # images[i] = edging


def isPoly(formatted_monopoly_data):
    res = np.zeros(len(formatted_monopoly_data))
    for i in range(0, len(formatted_monopoly_data)):
        res[i] = int(formatted_monopoly_data[i][1])
    
    return res

def getArgMax(probabilities):
    res = np.zeros(len(probabilities))
    for i in range(0, len(probabilities)):
        res[i] = np.argmax(probabilities[i])

    return res


In [8]:
# MAKE BIG TRAINING AND TESTING SET
images_3 = np.dstack([images] * 3)
images_3 = np.reshape(images_3, (-1, 300, 300, 3))

probs_formatted = probs
probs_label_encoder = LabelEncoder()
probs_integer = probs_label_encoder.fit_transform(probs_formatted)

probs_formatted = keras.utils.to_categorical(probs_integer)

types_formatted = types
types_label_encoder = LabelEncoder()
types_formatted = types_label_encoder.fit_transform(types_formatted)
types_formatted = keras.utils.to_categorical(types_formatted)

images_3_mono = images_3[types == "mono"]
images_3_poly = images_3[types == "poly"]
probs_formatted_mono = probs_formatted[types == "mono"]
probs_formatted_poly = probs_formatted[types == "poly"]

# Format "types_formatted" into "types_formatted | probs" for splitting data
types_probs_formatted = np.zeros((len(types_formatted), len(types_formatted[0]) + len(probs_formatted[0])))
for i in range (0, len(types_probs_formatted)):
    # types_probs_formatted[i] = np.concatenate(types_formatted[i], probs_formatted[i])
    types_probs_formatted[i, 0:len(types_formatted[0]):1] = types_formatted[i]
    types_probs_formatted[i, len(types_formatted[0]):len(types_probs_formatted[0]):1] = probs_formatted[i]


# training and testing data for types and final classifiers:
#   split types into test and train
#   x_train, test
#   y_mp_train, test
#   y_probs_train, test
x_train, x_test, y_combo_train, y_combo_test = train_test_split(images_3, types_probs_formatted, test_size=0.25, random_state=690)

y_mp_train = y_combo_train[:, 0:2]
y_train = y_combo_train[:, 2:6]

y_mp_test = y_combo_test[:, 0:2]
y_test = y_combo_test[:, 2:6]

In [9]:
# do the same with VGG, but for mono and poly images
vgg19_base = keras.applications.VGG19(weights='imagenet', include_top=False, input_shape=(300, 300, 3))
vgg19_base.trainable = False

vgg19_model = keras.models.Sequential([
  vgg19_base,
  keras.layers.GlobalAveragePooling2D(),
  keras.layers.Flatten(input_shape=vgg19_base.output_shape[1:]),
  keras.layers.Dense(4096, activation='relu', kernel_initializer='he_normal'),
  keras.layers.Dense(2048, activation='relu', kernel_initializer='he_normal'),
  keras.layers.Dense(len(np.unique(probs)), activation='softmax')
])

vgg19_model.compile(optimizer=keras.optimizers.Adam(learning_rate = 0.001, beta_1 = 0.9, beta_2 = 0.999, epsilon=1e-08),
              loss="categorical_crossentropy",
              metrics=['accuracy'])

es = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 3)

history_mono = vgg19_model.fit(x_train, y_train, epochs=10, validation_split = 0.2, callbacks=[es], batch_size=16)

# PATH = 'models/vgg19_all_model.keras'
# vgg19_model.save(PATH)

Train on 1574 samples, validate on 394 samples
Epoch 1/10
1574/1574 [==============================] - 14s 9ms/sample - loss: 6.5184 - acc: 0.6042 - val_loss: 1.1947 - val_acc: 0.5736
Epoch 2/10
1574/1574 [==============================] - 13s 9ms/sample - loss: 0.8944 - acc: 0.7084 - val_loss: 0.8146 - val_acc: 0.7360
Epoch 3/10
1574/1574 [==============================] - 14s 9ms/sample - loss: 0.7118 - acc: 0.7503 - val_loss: 0.7605 - val_acc: 0.7437
Epoch 4/10
1574/1574 [==============================] - 14s 9ms/sample - loss: 0.6048 - acc: 0.7795 - val_loss: 0.7480 - val_acc: 0.7386
Epoch 5/10
1574/1574 [==============================] - 14s 9ms/sample - loss: 0.5593 - acc: 0.7992 - val_loss: 0.8055 - val_acc: 0.7157
Epoch 6/10
1574/1574 [==============================] - 14s 9ms/sample - loss: 0.5371 - acc: 0.8088 - val_loss: 0.8844 - val_acc: 0.7284
Epoch 7/10
1574/1574 [==============================] - 14s 9ms/sample - loss: 0.5236 - acc: 0.8056 - val_loss: 1.0855 - val_acc: 0

In [7]:
# score = vgg19_model.evaluate(x_test, y_test)
result = vgg19_model.predict(x_test)
# print(dict(zip(vgg19_model.metrics_names, score)))
# print(vgg19_model.metrics_names)

def getArgMax(probabilities):
    res = np.zeros(len(probabilities))
    for i in range(0, len(probabilities)):
        res[i] = np.argmax(probabilities[i])

    return res


predicted_int = getArgMax(result)
y_test_int = getArgMax(y_test)

from sklearn.metrics import classification_report
print(classification_report(predicted_int, y_test_int))

              precision    recall  f1-score   support

         0.0       0.85      0.80      0.82       402
         1.0       0.49      0.38      0.43        90
         2.0       0.04      1.00      0.07         1
         3.0       0.73      0.82      0.77       163

    accuracy                           0.74       656
   macro avg       0.53      0.75      0.52       656
weighted avg       0.77      0.74      0.75       656



In [12]:
import performancetest as pt

y_true_m = y_test_int[isPoly(y_mp_test) == "mono"]
y_true_p = y_test_int[isPoly(y_mp_test) == "poly"]
y_predict_m = predicted_int[isPoly(y_mp_test) == "mono"]
y_predict_p = predicted_int[isPoly(y_mp_test) == "poly"]

pt.display_results(y_true_m, y_true_p, y_predict_m, y_predict_p)

c:\Users\jaych\anaconda3\envs\tensorflow-directml\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  This is separate from the ipykernel package so we can avoid doing imports until
c:\Users\jaych\anaconda3\envs\tensorflow-directml\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  after removing the cwd from sys.path.
c:\Users\jaych\anaconda3\envs\tensorflow-directml\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """
c:\Users\jaych\anaconda3\envs\tensorflow-directml\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise compari